In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
#import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
import pickle
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torchvision.models import resnet34
from helper_functions import train_or_load, train_loop

In [2]:
#load the shadow model trained in the other python script
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # sets to gpu if you have one
NUM_WORKERS = 4
resnet_shadow = resnet34(weights = None,num_classes = 200).to(DEVICE) #resnet_target is the shadow model
resnet_cifar = torch.load("shadow_models/resnet34_shadow_tinyimage_overtrained.pth",map_location=DEVICE)
resnet_shadow.load_state_dict(resnet_cifar)

<All keys matched successfully>

In [3]:
DATA_PATH = 'pickle/tinyimagenet/resnet34/shadow.p'
# Change the DATA_PATH to your local pickle file path


with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)


#splitting
#only use train set here
train_data, val_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
dataloader = torch.utils.data.DataLoader(
    train_data, batch_size=
     1 , shuffle=False, num_workers=NUM_WORKERS)
testloader =  torch.utils.data.DataLoader(val_data, batch_size=1,
                                          shuffle=True, num_workers=NUM_WORKERS)

for batch_idx, (img, label) in enumerate(dataloader):
    img = img.to(DEVICE)


In [4]:
#generate dataset for attack model
resnet_shadow.eval()
dataset_attack = []
with torch.no_grad():
    for images, labels in testloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(DEVICE), labels.to(DEVICE)
            
            # Forward pass
        logits = resnet_shadow(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values
        top_values, indices = torch.sort(top_values, dim=1, descending=True)
        dataset_attack.append([top_values,0])
        
with torch.no_grad():
    for images, labels in dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(DEVICE), labels.to(DEVICE)
            
            # Forward pass
        logits = resnet_shadow(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values
        top_values, indices = torch.sort(top_values, dim=1, descending=True)

        dataset_attack.append([top_values,1])
        



In [5]:


# Convert all tensors to the same dtype first
tensors = [data[0].float() for data in dataset_attack]  # Ensure all tensors are Float type
all_data = torch.cat(tensors, dim=0)  # Concatenate all tensors

# Calculate mean and std
mean = all_data.mean(dim=0)
std = all_data.std(dim=0)

# Standardize data in the list
standardized_data_list = [( (data[0] - mean) / std, data[1] ) for data in dataset_attack]


In [6]:
dataloader_attack = torch.utils.data.DataLoader(
    
    standardized_data_list, batch_size=32, shuffle=True, num_workers=NUM_WORKERS) #shuffled training data

In [7]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.bn1 = nn.BatchNorm1d(32)  # Matches the output of fc1
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)  # Matches the output of fc2
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)  # Matches the output of fc3
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)  # Matches the output of fc4
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)   # Matches the output of fc5
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x



In [8]:
model = SimpleNN()
criterion = nn.BCELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [9]:
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [14]:
# Define your training function

def train(model, dataloader, optimizer, criterion, epochs):
    for epoch in range(epochs):
        for inputs, labels in dataloader:
            inputs = inputs.float().to(DEVICE)  # Ensures input tensors are floats
            labels = labels.float().to(DEVICE).view(-1, 1)  # Ensures labels are floats and reshaped correctly

            optimizer.zero_grad()
            outputs = model(inputs.squeeze(dim = 1))
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")

In [15]:
#dict =  torch.load('attack_model.pth_3', map_location='cpu')
#model.load_state_dict(dict)

In [16]:
model.train()
model.to(DEVICE)
train(model,dataloader_attack,optimizer,criterion,20)
# train_or_load(model, dataloader_attack, optimizer, criterion, epochs=20, save_path='attack_models/attack_resnet_tinyimage.pth', device=DEVICE)
# train_loop(num_epochs=20, model=model, dataloader=dataloader_attack, optimizer=optimizer, criterion=criterion, device=DEVICE)

In [ ]:
torch.save(model.state_dict(), 'attack_models/attack_resnet_tinyimage.pth')

<h1>Evaluation</h1>

In [ ]:
DATA_PATH = "pickle/tinyimagenet/resnet34/eval.p"

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
# Convert all tensors to the same dtype first

dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=1 , shuffle=False, num_workers=1)
#splitting
for batch_idx, (img, label, membership) in enumerate(dataloader):
    img = img.to(DEVICE)


# Define your evaluation function


In [ ]:
dataset = []
resnet_shadow.eval()
with torch.no_grad():
    for images,_, b in dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(DEVICE), labels.to(DEVICE)
            
            # Forward pass
        logits = resnet_shadow(images)
        print(logits)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values
        

        dataset.append([top_values,b])

tensor([[-1.4187e+00, -6.2129e+00,  5.6130e-01,  4.1395e-01, -3.1399e-01,
          4.3520e-01, -3.1133e+00, -2.4124e+00, -2.1941e+00,  5.9890e-01,
         -8.3711e-01, -1.6750e+00,  1.0958e+00,  1.9525e+00,  2.8084e+00,
          4.1047e+00,  1.8432e+00,  2.3254e+00, -3.2230e+00, -2.6141e+00,
         -1.3597e+00, -6.1866e-01,  1.0660e+00,  3.6795e+00, -1.7001e+00,
          2.4174e+00,  4.1679e-01,  6.5097e-01, -1.4478e+00,  2.2793e-03,
          1.3609e+00,  1.8928e+00, -9.1237e-01,  2.1235e+00,  3.4328e+00,
          2.8120e+00, -6.7081e-02,  7.1951e-01, -8.4047e-01, -6.7975e-01,
          7.5848e-01, -1.1356e+00,  1.9308e+00,  6.1964e-02, -5.5289e+00,
          3.4242e-01,  3.1275e+00,  2.0492e+00,  4.5044e-01, -1.2693e+00,
         -8.0436e-01, -1.5231e+00, -3.6712e+00, -6.5417e-01,  5.3713e+00,
          3.6178e-01,  2.1209e+00, -1.6241e+00, -1.9321e-01, -1.7070e+00,
         -6.2005e+00, -7.4738e+00, -2.0272e+00, -4.1228e+00, -2.4964e+00,
         -5.1331e+00, -6.1966e+00, -2.

In [ ]:
resnet_target_model = resnet34(pretrained = False,num_classes = 200).to(DEVICE)
check = torch.load("models/resnet34_tinyimagenet.pth", map_location=DEVICE)
resnet_target_model.load_state_dict(check["net"])
resnet_target_model.eval()

c:\Users\Gleb\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Gleb\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
#get posteriors from target
dataset_eval = []
with torch.no_grad():
    for images,_, member in dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(DEVICE), labels.to(DEVICE)
            
            # Forward pass
        logits = resnet_target_model(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values #order poseri
        sorted_tensor, indices = torch.sort(top_values, dim=1,descending=True)
        dataset_eval.append([sorted_tensor, member.item()])
        


In [ ]:
# Convert all tensors to the same dtype first
tensors = [data[0].float() for data in dataset_eval]  # Ensure all tensors are Float type
all_data = torch.cat(tensors, dim=0)  # Concatenate all tensors

# Calculate mean and std
mean = all_data.mean(dim=0)
std = all_data.std(dim=0)

# Standardize data in the list
dataset = [( (data[0] - mean) / std, data[1] ) for data in dataset_eval]

In [ ]:
dataloader_eval = torch.utils.data.DataLoader(
    # TODO: make variables less convoluted
    dataset, batch_size=1 , shuffle=False, num_workers=1)

In [ ]:
def evaluate(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for data in dataloader:
            inputs = data[0]
            
            labels = data[1].float()
            labels = torch.tensor(labels).to(DEVICE).float()
            #labels = torch.tensor(labels[0])
            optimizer.zero_grad()

            outputs = model(inputs.squeeze(dim=1))
            predicted = torch.round(outputs)  # Round the outputs to 0 or 1
            #predicted = abs(predicted -1)
            print(predicted,labels)

            total += labels.size(0)  # Increment the total count by batch size
            correct += (predicted == labels).sum().item()  # Count correct predictions

    accuracy = correct / total
    return accuracy

# Assuming you have a DataLoader called 'test_loader' for evaluation
# Create an instance of your model

# Assuming 'test_loader' is your DataLoader for evaluation
accuracy = evaluate(model, dataloader_eval)

print(f'Accuracy: {accuracy:.2f}')

C:\Users\Gleb\AppData\Local\Temp\ipykernel_17132\714747091.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(DEVICE).float()


tensor([[1.]], device='cuda:0') tensor([1.], device='cuda:0')
tensor([[1.]], device='cuda:0') tensor([1.], device='cuda:0')
tensor([[1.]], device='cuda:0') tensor([1.], device='cuda:0')
tensor([[1.]], device='cuda:0') tensor([1.], device='cuda:0')
tensor([[1.]], device='cuda:0') tensor([1.], device='cuda:0')
tensor([[1.]], device='cuda:0') tensor([1.], device='cuda:0')
tensor([[1.]], device='cuda:0') tensor([1.], device='cuda:0')
tensor([[1.]], device='cuda:0') tensor([1.], device='cuda:0')
tensor([[1.]], device='cuda:0') tensor([1.], device='cuda:0')
tensor([[1.]], device='cuda:0') tensor([1.], device='cuda:0')
tensor([[1.]], device='cuda:0') tensor([1.], device='cuda:0')
tensor([[1.]], device='cuda:0') tensor([1.], device='cuda:0')
tensor([[1.]], device='cuda:0') tensor([1.], device='cuda:0')
tensor([[1.]], device='cuda:0') tensor([1.], device='cuda:0')
tensor([[1.]], device='cuda:0') tensor([1.], device='cuda:0')
tensor([[1.]], device='cuda:0') tensor([1.], device='cuda:0')
tensor([

In [ ]:
#1 - 0.26 = 0.74 > 0.68